In [1]:
import geopandas as gpd
import momepy as mm
from tqdm import tqdm
from momepy import limit_range
import numpy as np
import pandas as pd
from pysal.explore.inequality.theil import Theil
import libpysal
import scipy as sp

/Users/martin/anaconda3/envs/geo_dev/lib/python3.8/site-packages/pysal/explore/segregation/network/network.py:15: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  warn(
/Users/martin/anaconda3/envs/geo_dev/lib/python3.8/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [2]:
gdf = pd.read_csv('/Users/martin/Dropbox/Academia/Data/Geo/Prague/Clustering/primary.csv', index_col=0)

In [3]:
spatial_weights = libpysal.io.open('/Users/martin/Dropbox/Academia/Data/Geo/Prague/Clustering/queen3_uID.gal', 'r').read()

/Users/martin/anaconda3/envs/geo_dev/lib/python3.7/site-packages/libpysal/weights/weights.py:165: UserWarning: The weights matrix is not fully connected: 
 There are 128 disconnected components.
  warnings.warn(message)


In [1]:
characters = ['stcOri', 'sdcLAL', 'sdcAre', 'sscCCo', 'sscERI',
       'stcSAl', 'sicCAR', 'sicFAR', 'mtcWNe', 'mdcAre', 'licGDe', 'ltcWRB',
       'sdbHei', 'sdbAre', 'sdbVol', 'sdbPer', 'sdbCoA',
       'ssbFoF', 'ssbVFR', 'ssbCCo', 'ssbCor', 'ssbSqu', 'ssbERI', 'ssbElo',
       'ssbCCM', 'ssbCCD', 'stbOri', 'stbSAl', 'stbCeA', 'mtbSWR',
       'mtbAli', 'mtbNDi', 'libNCo', 'ldbPWL', 'ltbIBD', 'ltcBuA', 'mtdDeg',
       'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo', 'ldsCDL', 'xcnSCl',
       'mtdMDi', 'lddNDe', 'linWID', 'lddRea', 'lddARe', 'sddAre', 'midRea',
       'midAre', 'sdsLen', 'sdsSPW', 'sdsSPH', 'sdsSPR', 'sdsSPO', 'sdsSWD',
       'sdsSHD', 'sssLin', 'sdsAre', 'sisBpM', 'misRea', 'mdsAre', 'ldsMSL', 'ldsRea', 'ldkAre',
       'ldkPer', 'lskCCo', 'lskERI', 'lskCWA', 'ltkOri', 'ltkWNB', 'likWBB']

In [2]:
len(characters)

74

In [4]:
fixTheil = ['sdbCoA', 'mtbSWR', 'mtbNDi',
       'libNCo', 'ltbIBD', 'lcdMes',
       'linP3W', 'linP4W', 'linPDE',
       'lcnClo', 'ldsCDL', 'xcnSCl',
       'lddNDe', 'linWID', 'sdsSPH',
       'sdsSPR', 'sdsSPO', 'sdsSWD',
       'sdsSHD', 'sssLin', 'ltkWNB']

In [5]:
means = {}
ranges = {}
theils = {}

In [6]:
for ch in fixTheil:
    means[ch] = []
    ranges[ch] = []
    theils[ch] = []

In [7]:
unique_id = 'uID'

In [8]:
spatial_weights.neighbors = {int(k): [int(i) for i in v] for k, v in spatial_weights.neighbors.items()}

In [9]:
gdf.set_index('uID', inplace=True)

In [10]:
gdf['lcdMes'] = gdf.apply(
            lambda row: row.lcdMes if row.lcdMes >= 0 else 0,
            axis=1,
        )

In [164]:
gdf

,uID,bID,stcOri,sdcLAL,sdcAre,sscCCo,sscERI,stcSAl,sicCAR,sicFAR,...,ldsMSL,ldsRea,ldkAre,ldkPer,lskCCo,lskERI,lskCWA,ltkOri,ltkWNB,likWBB
0,0,0,7.603973,33.817996,342.233358,0.381010,0.891051,7.787476,0.087768,0.614375,...,500.099794,1.006508e+06,397667.372035,3615.243681,0.404219,0.724434,880.163669,28.214414,0.000553,0.000108
1,1,1,3.712491,58.703914,1435.882458,0.530512,0.935147,2.259805,0.598660,2.993299,...,129.395330,3.504396e+05,262570.110986,2902.374890,0.392367,0.742951,714.928910,8.174868,0.003790,0.000369
2,2,4,12.121107,34.814040,433.237769,0.455122,0.940945,14.950339,0.108374,0.108374,...,245.801133,4.159394e+05,56345.646398,1310.808733,0.264617,0.799244,389.760230,28.532001,0.004577,0.000461
3,3,5,7.700002,45.376335,839.521945,0.519139,0.994977,5.437630,0.096908,0.290723,...,334.823124,1.586278e+06,93980.927007,1236.106936,0.694690,0.994124,119.993680,6.159727,0.004045,0.000638
4,4,6,29.080840,135.981870,7058.965193,0.486059,1.039719,27.965128,0.017193,0.017193,...,133.672812,3.444156e+05,7058.965193,328.785377,0.486059,1.039719,31.062697,29.080840,0.006083,0.000142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140310,140457,5488,28.178572,98.965803,3505.778714,0.455748,0.920684,6.799673,0.573294,4.013055,...,117.971921,3.082993e+05,56606.217751,951.571799,0.596143,1.009986,94.926096,35.086045,0.008407,0.000654
140311,140458,1139,24.776814,92.897524,5114.085960,0.754518,1.071859,12.900064,0.155292,0.310584,...,129.984846,4.016710e+05,40933.111154,1380.026247,0.224597,0.664578,447.681498,10.487112,0.005797,0.000220
140312,140459,4960,15.112430,34.559509,520.511963,0.554888,0.948496,1.076186,0.409776,2.458657,...,93.310638,1.411415e+05,9762.288285,426.012538,0.550692,0.956055,61.986071,4.259050,0.011737,0.002663
140313,140460,4960,20.447865,19.148696,193.498945,0.671909,1.019082,6.411622,0.421273,2.527640,...,93.310638,1.411415e+05,9762.288285,426.012538,0.550692,0.956055,61.986071,4.259050,0.011737,0.002663


In [11]:
def theil(y):
    y = np.array(y)
    n = len(y)
    plus = y + np.finfo('float').tiny * (y == 0)  # can't have 0 values
    yt = plus.sum(axis=0)
    s = plus / (yt * 1.0)
    lns = np.log(n * s)
    slns = s * lns
    t = sum(slns)
    return t

for index, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
    neighbours = spatial_weights.neighbors[index].copy()
    neighbours.append(index)
    
    for ch in characters:
        values_list = gdf.loc[neighbours][ch]        
        #idec = limit_range(values_list.tolist(), rng=(10, 90))
        #mean
        #means[ch].append(np.mean(values_list))
        #range
        ranges[ch].append(sp.stats.iqr(values_list, rng=(25, 75)))
        #theil
        #theils[ch].append(theil(idec))
        
        


100%|██████████| 140315/140315 [1:24:54<00:00, 27.54it/s]  


In [2]:
data = pd.read_csv('/Users/martin/Dropbox/Academia/Data/Geo/Prague/Clustering/patterned.csv', index_col=0)

In [3]:
data

,stcOri_meanIQ3,stcOri_rangeIQ3,stcOri_theilID3,sdcLAL_meanIQ3,sdcLAL_rangeIQ3,sdcLAL_theilID3,sdcAre_meanIQ3,sdcAre_rangeIQ3,sdcAre_theilID3,sscCCo_meanIQ3,...,ltkOri_rangeIQ3,ltkOri_theilID3,ltkWNB_meanIQ3,ltkWNB_rangeIQ3,ltkWNB_theilID3,likWBB_meanIQ3,likWBB_rangeIQ3,likWBB_theilID3,uID,sdbHei_Wmean3
0,16.752005,22.129068,0.319261,116.591871,126.024037,0.148450,7396.442162,11944.895882,0.369595,0.486557,...,0.000000,2.220446e-16,0.000553,0.000000,-2.220446e-16,0.000108,0.000000,4.440892e-16,0,18.197127
1,14.397474,13.068941,0.170276,54.212949,30.306644,0.062006,1845.916602,993.765713,0.361074,0.433156,...,11.554043,1.647543e-01,0.006354,0.003273,3.731938e-02,0.001060,0.000892,1.433776e-01,1,14.688127
2,24.016568,9.287744,0.027263,66.619067,34.828700,0.057893,1685.808768,1152.752592,0.109392,0.461839,...,5.143269,1.755450e-02,0.004802,0.000000,1.925702e-01,0.000503,0.000000,8.601720e-02,2,6.347597
3,13.777194,14.149229,0.175929,67.561781,21.585726,0.036034,1872.938586,1099.398404,0.098376,0.479430,...,1.136783,5.191117e-03,0.003129,0.002236,7.465612e-02,0.000428,0.000380,8.478286e-02,3,9.026955
4,12.730854,14.990882,0.220775,105.477323,77.091866,0.056973,4638.046195,5431.598741,0.210759,0.443735,...,7.770669,1.780352e-01,0.006669,0.002913,4.561240e-02,0.000465,0.000616,2.198603e-01,4,16.370168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140310,35.359461,3.636083,0.002019,55.617147,42.056210,0.087692,1365.134317,1421.333332,0.264876,0.459145,...,2.024198,3.149134e-04,0.009520,0.003151,1.563030e-02,0.000884,0.000246,4.699536e-02,140457,21.991471
140311,18.449721,17.294178,0.128390,73.541448,40.024503,0.060183,2742.632159,1599.818377,0.272273,0.464714,...,29.928170,1.279961e-01,0.007942,0.002874,1.262205e-01,0.000900,0.000437,3.716466e-01,140458,7.965448
140312,12.183539,11.654825,0.253338,32.772448,12.975189,0.018948,465.755253,346.185461,0.074041,0.518240,...,8.885753,2.759707e-01,0.012258,0.001082,6.406237e-03,0.002189,0.001076,4.200610e-02,140459,17.905549
140313,13.056166,13.621215,0.335320,32.620964,14.029790,0.025162,448.670786,293.817507,0.107598,0.502563,...,1.095831,1.862364e-01,0.013092,0.002624,9.151888e-03,0.002379,0.000325,7.250857e-03,140460,16.935371


In [4]:
data_p = pd.read_csv('/Users/martin/Dropbox/Academia/Data/Geo/Prague/Clustering/patterned_data.csv', index_col=0)

In [5]:
data_p

,stcOri_meanIQ3,stcOri_rangeIQ3,stcOri_theilID3,sdcLAL_meanIQ3,sdcLAL_rangeIQ3,sdcLAL_theilID3,sdcAre_meanIQ3,sdcAre_rangeIQ3,sdcAre_theilID3,sscCCo_meanIQ3,...,lcnClo_simpson,lddRea_simpson,sdsSPW_simpson,sdsSPO_simpson,sdsSWD_simpson,sssLin_simpson,lskCCo_simpson,lskERI_simpson,ltkOri_simpson,ltkWNB_simpson
0,16.752005,22.129068,0.319261,116.591871,126.024037,0.148450,7396.442162,11944.895882,0.369595,0.486557,...,1.000000,0.814507,0.505351,1.000000,0.505351,0.505351,1.000000,1.000000,1.000000,1.000000
1,14.397474,13.068941,0.170276,54.212949,30.306644,0.062006,1845.916602,993.765713,0.361074,0.433156,...,0.337604,0.308864,0.567175,0.538089,0.266274,0.765928,0.401662,0.370845,0.267659,0.372230
2,24.016568,9.287744,0.027263,66.619067,34.828700,0.057893,1685.808768,1152.752592,0.109392,0.461839,...,1.000000,0.415556,0.326667,0.642222,0.440000,1.000000,0.415556,0.460000,0.397778,0.397778
3,13.777194,14.149229,0.175929,67.561781,21.585726,0.036034,1872.938586,1099.398404,0.098376,0.479430,...,0.541728,0.512099,0.311605,0.306667,0.323457,0.737284,0.368889,0.368889,0.429136,0.429136
4,12.730854,14.990882,0.220775,105.477323,77.091866,0.056973,4638.046195,5431.598741,0.210759,0.443735,...,0.268861,0.266804,0.210562,0.314815,0.242112,0.659122,0.283265,0.294239,0.290809,0.331276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140310,35.359461,3.636083,0.002019,55.617147,42.056210,0.087692,1365.134317,1421.333332,0.264876,0.459145,...,0.582766,0.292880,0.353288,0.447800,0.387392,1.000000,0.526531,0.565351,0.591837,0.392290
140311,18.449721,17.294178,0.128390,73.541448,40.024503,0.060183,2742.632159,1599.818377,0.272273,0.464714,...,0.276008,0.529517,0.213188,0.242147,0.341501,0.529962,0.267988,0.229227,0.259969,0.304077
140312,12.183539,11.654825,0.253338,32.772448,12.975189,0.018948,465.755253,346.185461,0.074041,0.518240,...,0.451250,0.405000,0.533750,0.511250,0.371250,1.000000,0.441250,0.463750,0.531250,0.463750
140313,13.056166,13.621215,0.335320,32.620964,14.029790,0.025162,448.670786,293.817507,0.107598,0.502563,...,0.401860,0.387397,0.552686,0.509298,0.311983,1.000000,0.410124,0.627066,0.648760,0.451446


In [13]:
for ch in fixTheil:
    data[ch + '_theilID3'] = theils[ch]

In [14]:
data.isna().any().any()

False

In [ ]:
data.to_csv('/Users/martin/Dropbox/Academia/Data/Geo/Prague/Clustering/patterned.csv')

In [129]:
nans

['lcdMes_theilID3']

In [132]:
data.loc[data['lcdMes_theilID3'].isna()].uID

551          552
608          609
609          610
1249        1250
3849        3854
           ...  
128864    128983
134535    134675
137935    138078
138546    138690
139589    139734
Name: uID, Length: 112, dtype: int64

In [28]:
gdf.to_csv('/Users/martin/Dropbox/Academia/Data/Geo/Prague/Clustering/primary+patterned.csv')

In [146]:
neighbours = spatial_weights.neighbors[552].copy()
neighbours.append(552)

In [147]:
values_list = gdf.loc[neighbours]['lcdMes']        
idec = limit_range(values_list.tolist(), rng=(10, 90))

In [148]:
max(idec)

0.15151515151515152

In [149]:
y = np.array(idec)
n = len(y)
plus = y + np.finfo('float').tiny * (y == 0)  # can't have 0 values
yt = plus.sum(axis=0)
s = plus / (yt * 1.0)
lns = np.log(n * s)
slns = s * lns
t = sum(slns)

In [150]:
t

nan

In [151]:
values_list

uID
49152     0.151515
107016   -0.333333
112650    0.111111
98316     0.103093
85005     0.151515
            ...   
33259     0.111111
494       0.151515
100080    0.076923
125694    0.111111
552       0.103093
Name: lcdMes, Length: 71, dtype: float64

In [31]:
suffixes = ['IQ3', 'ID3']
patterned = [x for x in gdf.columns if 'IQ3' in x or 'ID3' in x]

In [33]:
patterned.append('uID')

In [45]:
gdf.reset_index()[patterned].to_csv('/Users/martin/Dropbox/Academia/Data/Geo/Prague/Clustering/patterned.csv')

# Weighted characters

In [39]:
weigh = ['sdbHei', 'ssbFoF', 'ssbVFR', 'ssbCCo', 'ssbCor', 'ssbSqu', 'ssbERI']
results = {}
for w in weigh:
    results[w] = []


for index, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
    neighbours = spatial_weights.neighbors[index]

    if neighbours:
        building_neighbours = gdf.loc[neighbours]
        
        for w in weigh:
            
            results[w].append(
                (
                    sum(building_neighbours[w] * building_neighbours['sdbAre'])
                    + (row[w] * row['sdbAre'])
                )
                / (sum(building_neighbours['sdbAre']) + row['sdbAre'])
            )


100%|██████████| 140315/140315 [07:48<00:00, 299.73it/s]


In [40]:
for w in weigh:
    gdf[w + '_Wmean3'] = results[w]

In [41]:
weighted = [x for x in gdf.columns if '_Wmean3' in x]

In [44]:
patterned = patterned + weighted

In [46]:
gdf['ssbFoF_w'] = gdf.ssbFoF * gdf.sdbAre

In [50]:
#gdf.reset_index(inplace=True)
gdf['ssbFoF_WmeanIQ3'] = mm.average_character(gdf, 'ssbFoF_w', spatial_weights, 'uID', rng=(25, 75))

Calculating average character value...



100%|██████████| 140315/140315 [06:39<00:00, 351.07it/s]


Average character value calculated.


In [54]:
gdf[patterned[:-6]].to_csv('/Users/martin/Dropbox/Academia/Data/Geo/Prague/Clustering/patterned.csv')

In [53]:
patterned[:-6]

['stcOri_meanIQ3',
 'stcOri_rangeIQ3',
 'stcOri_theilID3',
 'sdcLAL_meanIQ3',
 'sdcLAL_rangeIQ3',
 'sdcLAL_theilID3',
 'sdcAre_meanIQ3',
 'sdcAre_rangeIQ3',
 'sdcAre_theilID3',
 'sscCCo_meanIQ3',
 'sscCCo_rangeIQ3',
 'sscCCo_theilID3',
 'sscERI_meanIQ3',
 'sscERI_rangeIQ3',
 'sscERI_theilID3',
 'stcSAl_meanIQ3',
 'stcSAl_rangeIQ3',
 'stcSAl_theilID3',
 'sicCAR_meanIQ3',
 'sicCAR_rangeIQ3',
 'sicCAR_theilID3',
 'sicFAR_meanIQ3',
 'sicFAR_rangeIQ3',
 'sicFAR_theilID3',
 'mtcWNe_meanIQ3',
 'mtcWNe_rangeIQ3',
 'mtcWNe_theilID3',
 'mdcAre_meanIQ3',
 'mdcAre_rangeIQ3',
 'mdcAre_theilID3',
 'licGDe_meanIQ3',
 'licGDe_rangeIQ3',
 'licGDe_theilID3',
 'ltcWRB_meanIQ3',
 'ltcWRB_rangeIQ3',
 'ltcWRB_theilID3',
 'sdbHei_meanIQ3',
 'sdbHei_rangeIQ3',
 'sdbHei_theilID3',
 'sdbAre_meanIQ3',
 'sdbAre_rangeIQ3',
 'sdbAre_theilID3',
 'sdbVol_meanIQ3',
 'sdbVol_rangeIQ3',
 'sdbVol_theilID3',
 'sdbPer_meanIQ3',
 'sdbPer_rangeIQ3',
 'sdbPer_theilID3',
 'sdbCoA_meanIQ3',
 'sdbCoA_rangeIQ3',
 'sdbCoA_theilID3

In [173]:
len(theils[ch])

281244